In [25]:
from tensorflow.keras.models import model_from_json
import numpy as np

import tensorflow as tf


class FacialExpressionModel(object):

    EMOTIONS_LIST = ["Angry", "Disgust",
                    "Fear", "Happy",
                    "Neutral", "Sad",
                    "Surprise"]

    def __init__(self, model_json_file, model_weights_file):
        # load model from JSON file
        with open(model_json_file, "r") as json_file:
            loaded_model_json = json_file.read()
            self.loaded_model = model_from_json(loaded_model_json)

        # load weights into the new model
        self.loaded_model.load_weights(model_weights_file)
        self.loaded_model._make_predict_function()

    def predict_emotion(self, img):
        self.preds = self.loaded_model.predict(img)
        return FacialExpressionModel.EMOTIONS_LIST[np.argmax(self.preds)]


## Getting frames and doing prediction

In [26]:
  
import cv2
import numpy as np

facec = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
model = FacialExpressionModel("./model.json", "./model_weights.h5")
font = cv2.FONT_HERSHEY_SIMPLEX

ret=[]

class VideoCamera(object):
    def __init__(self):
        #self.video = cv2.VideoCapture(r"D:\Meh\College\Mini project\Facial-Expression-Recognition-with-CNNs-master\myenv\vid\Video For Practicing Eye Contact.mp4")
        #self.video = cv2.VideoCapture(r"A:/project/vid/The secrets to decoding facial expressions.mp4")#change the backward slashes to forward slashes to run in vscode
        self.video = cv2.VideoCapture(0)
        
    def __del__(self):
        self.video.release()

    # returns camera frames along with bounding boxes and predictions
    def get_frame(self):
        _, fr = self.video.read()
        gray_fr = cv2.cvtColor(fr, cv2.COLOR_BGR2GRAY)
        faces = facec.detectMultiScale(gray_fr, 1.3, 5)

        for (x, y, w, h) in faces:
            fc = gray_fr[y:y+h, x:x+w]
            roi = cv2.resize(fc, (48, 48))
            pred = model.predict_emotion(roi[np.newaxis, :, :, np.newaxis])
            ret.append(pred)
            
            cv2.putText(fr, pred, (x, y), font, 1, (255, 255, 0), 2)
            cv2.rectangle(fr,(x,y),(x+w,y+h),(255,0,0),2)

        return fr

## Function for showing output video

In [27]:
def gen(camera):
    while True:
        frame = camera.get_frame()
        cv2.imshow('Facial Expression Recognization',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()

## Running the code

In [28]:
gen(VideoCamera())

In [29]:
max_res = max(ret,key=ret.count)
print("The maximum shown emotion is: ", max_res)

The maximum shown emotion is:  Neutral


In [30]:
import random
import os
import pathlib
import subprocess

mp = 'C:/Program Files (x86)/Windows Media Player/wmplayer.exe'
if max_res == 'Angry':
    randomfile = random.choice(os.listdir(os.path.abspath('')+"\\songs\\Angry\\"))
    print('You are angry !!!! please calm down:) ,I will play song for you :' + randomfile)
    file = (os.path.abspath('')+"\\songs\\Angry\\" + randomfile)
    subprocess.call([mp, file])

if max_res == 'Happy':
    randomfile = random.choice(os.listdir(os.path.abspath('')+"\\songs\\Happy\\"))
    print('You are smiling :) ,I playing special song for you: ' + randomfile)
    file = (os.path.abspath('')+"\\songs\\Happy\\" + randomfile)
    subprocess.call([mp, file])

if max_res == 'Fear':
    randomfile = random.choice(os.listdir(os.path.abspath('')+"\\songs\\Fear\\"))
    print('You have fear of something ,I playing song for you: ' + randomfile)
    file = (os.path.abspath('')+"\\songs\\Fear\\"+ randomfile)
    subprocess.call([mp, file])

if max_res == 'Neutral':
    randomfile = random.choice(os.listdir(os.path.abspath('')+"\\songs\\Neutral\\"))
    print('You look good , hear to this song:  ' + randomfile)
    file = (os.path.abspath('')+"\\songs\\Neutral\\" + randomfile)
    subprocess.call([mp, file])

You look good , hear to this song:  Pirates Of The Caribbean Orchestral Medley - FL Studio Remake.mp3
